In [1]:
import tensorflow as tf
sys.path.append('../')
sys.path.append('../..')
import experiment_utils as eu
import sys
from data_processing.cifar10 import get_cifar10
import importlib
import config as conf
import models.model as mod


In [10]:
import models.model as mod
importlib.reload(mod)
config = conf.get_byol()
cn = mod.ClassificationNetwork(
    saved_encoder_path='saved_model/encoder.h5',
    saved_projection_path='saved_model/projection.h5',
    config=config)


In [11]:
cn.projection_head.weights

[<tf.Variable 'mlp_head_4/dense_11/kernel:0' shape=(512, 512) dtype=float32, numpy=
 array([[-0.03366643,  0.07453532, -0.06991264, ..., -0.04364319,
          0.05584826, -0.01795833],
        [ 0.05009131,  0.03266796,  0.06576122, ..., -0.02965577,
          0.04522863,  0.05607836],
        [-0.06709336,  0.06006589,  0.04476017, ..., -0.04056853,
         -0.01472719, -0.06225573],
        ...,
        [-0.06842595,  0.01187088, -0.07315741, ...,  0.04633082,
         -0.0278316 , -0.04483537],
        [-0.02067022,  0.00354103,  0.01306467, ...,  0.04997816,
          0.0484774 ,  0.06361011],
        [-0.05471572, -0.03585834, -0.00244421, ...,  0.00994697,
          0.05691087, -0.03207906]], dtype=float32)>,
 <tf.Variable 'mlp_head_4/dense_11/bias:0' shape=(512,) dtype=float32, numpy=
 array([-3.96210398e-10,  1.04025483e-10, -4.99427610e-10,  9.76719955e-11,
        -9.10935161e-10, -4.33596492e-10,  8.41938241e-10,  9.84948234e-11,
         1.96739097e-10, -1.37295908e-09,  

In [30]:
ds, _ = get_cifar10(batch_size=config.batch_size, split='train')
ds_small = ds.take(15)
for d in ds.take(1):
    result = cn(d)

result

<tf.Tensor: shape=(128, 10), dtype=float32, numpy=
array([[-19.556858  ,   7.275477  , -10.091932  , ...,  -9.206974  ,
         -0.4547162 ,  20.913065  ],
       [-21.12404   ,   7.8499074 , -11.287449  , ...,  -9.874862  ,
         -0.5003481 ,  22.483677  ],
       [-21.444878  ,   7.236249  , -11.199618  , ..., -11.504858  ,
          0.41868877,  25.566057  ],
       ...,
       [-18.448132  ,   6.7379494 ,  -9.7340145 , ...,  -8.890999  ,
         -0.14152908,  20.107239  ],
       [-21.823511  ,   8.018632  , -11.156696  , ..., -11.597046  ,
          0.65368366,  25.391071  ],
       [-24.853655  ,   8.875578  , -13.3606825 , ..., -12.373997  ,
         -0.8188281 ,  27.734974  ]], dtype=float32)>

In [2]:
importlib.reload(eu)
config = conf.get_byol()
config.batch_size = 10
ds, _ = get_cifar10(batch_size=config.batch_size, split='train')
ds_small = ds.take(15)
experiment = eu.Experiment(config)
experiment.train(ds_small, 'saved_model', epochs=1)


AttributeError: "'eps'"

In [ ]:
experiment.online_network.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
res_net18 (ResNet18)         (None, 512)               11183232  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
mlp_head (MLPHead)           (None, 128)               329728    
_________________________________________________________________
mlp_head_1 (MLPHead)         (None, 128)               133120    
Total params: 11,646,080
Trainable params: 11,634,432
Non-trainable params: 11,648
_________________________________________________________________


In [ ]:
y_true = [[0., 1.], [1., 1.]]
y_pred = [[1., 0.], [1., 1.]]
# Using 'auto'/'sum_over_batch_size' reduction type.
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
cosine_loss